In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.metrics import mean_squared_error



# Normalize the data
scaler = MinMaxScaler()
quantity_scaled = scaler.fit_transform(df_weekly[["Quantity"]])

# Prepare data for GRU
X = []
y = []
seq_length = 10  # Adjust sequence length as needed

for i in range(len(quantity_scaled) - seq_length):
    X.append(quantity_scaled[i:i+seq_length])
    y.append(quantity_scaled[i+seq_length])

X = np.array(X)
y = np.array(y)

# Split data into train and test sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build GRU model
model = Sequential([
    GRU(50, activation='relu', input_shape=(seq_length, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=2)

# Forecast the next steps
forecast_steps = len(y_test)
forecast_scaled = model.predict(X_test)
forecast = scaler.inverse_transform(forecast_scaled)

# Plot the original data and the forecast
plt.figure(figsize=(10, 6))
plt.plot(df_weekly.index[-forecast_steps:], df_weekly["Quantity"][-forecast_steps:], label='Original Data')
plt.plot(df_weekly.index[-forecast_steps:], forecast, color='red', label='Forecast')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Quantity')
plt.title('GRU Model Forecasting for Quantity')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate RMSE
mse = mean_squared_error(df_weekly["Quantity"][-forecast_steps:], forecast)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')
